In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

d = pd.read_csv("framingham.csv").dropna()

X = d.drop(columns=["TenYearCHD"])
y = d["TenYearCHD"]

X = X.fillna(X.mean())

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

rf = RandomForestClassifier(class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

In [21]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd

style_bg = """

<style>
    .title {
        font-size: 24px;
        font-weight: bold;
        text-align: center;
        margin-bottom: 10px;
        color: #2c3e50;
    }
    .subtitle {
        font-size: 16px;
        text-align: center;
        margin-bottom: 20px;
        color: #34495e;
        font-style: italic;
    }
    body {
        background-color: #f0e6d2 !important; 
    }
    .custom-container {
         background-color: #c2a77a; 
         padding: 15px;
         border-radius: 10px;
         width: 90%;
         max-width: 850px;
         margin: auto;
         box-shadow: 0 4px 8px rgba(0,0,0,0.1);
         display: flex;
         flex-direction: column;
         align-items: center;
    }
    .custom-label {
        font-weight: bold;
        margin-right: 5px;
        width: 100px;
        display: inline-block;
        text-align: right;
    }
    .custom-widget {
        background-color: #ad856a; 
        border-radius: 5px;
        padding-left: 5px;
        width: 180px;
        color: black;
        font-weight: normal;
    }
    .predict-button {
        background-color: #ad856a !important;
        color: black !important;
        border-radius: 10px !important;
        width: 120px !important;
        margin-top: 10px !important;
    }
    .output-box {
        margin: 10px 5px;
        background-color: #ad856a;
        border-radius: 8px;
        padding: 10px;
        width: 1000px;
        min-height: 100px;
        color: black;
        font-weight: bold;
    }
    .form-items-container {
        width: 100%;
        display: flex;
        flex-direction: column;
        align-items: center;
    }
</style>
"""
title_html = """
<div class="title">Heart Health Prediction</div>
<div class="subtitle">Please enter the correct information for the below mentioned questions</div>
"""

display(HTML(style_bg))
display(HTML(title_html))


# Widgets setup (same as before)
gender = widgets.Dropdown(
    options=['Male', 'Female'],
    value='Male',
    layout=widgets.Layout(width='180px'),
)
age = widgets.BoundedIntText(value=0, min=0, max=120, layout=widgets.Layout(width='180px'))
currentSmoker = widgets.Dropdown(options=[('No (0)', 0), ('Yes (1)', 1)], value=0, layout=widgets.Layout(width='180px'))
cigsPerDay = widgets.BoundedIntText(value=0, min=0, max=100, layout=widgets.Layout(width='180px'))
BPMeds = widgets.Dropdown(options=[('No (0)', 0), ('Yes (1)', 1)], value=0, layout=widgets.Layout(width='180px'))
prevalentStroke = widgets.Dropdown(options=[('No (0)', 0), ('Yes (1)', 1)], value=0, layout=widgets.Layout(width='180px'))
prevalentHyp = widgets.Dropdown(options=[('No (0)', 0), ('Yes (1)', 1)], value=0, layout=widgets.Layout(width='180px'))
diabetes = widgets.Dropdown(options=[('No (0)', 0), ('Yes (1)', 1)], value=0, layout=widgets.Layout(width='180px'))
totChol = widgets.BoundedIntText(value=100, min=100, max=600, layout=widgets.Layout(width='180px'))
sysBP = widgets.BoundedIntText(value=80, min=40, max=300, layout=widgets.Layout(width='180px'))
diaBP = widgets.BoundedIntText(value=40, min=20, max=200, layout=widgets.Layout(width='180px'))
BMI = widgets.BoundedFloatText(value=10, min=10, max=80, layout=widgets.Layout(width='180px'))
heartRate = widgets.BoundedIntText(value=30, min=20, max=200, layout=widgets.Layout(width='180px'))
glucose = widgets.BoundedIntText(value=50, min=30, max=600, layout=widgets.Layout(width='180px'))

def make_row(label_text, widget):
    label = widgets.HTML(value=f'<b style="width:100px; display:inline-block; text-align:right;">{label_text}</b>')
    widget.add_class('custom-widget')
    return widgets.HBox([label, widget], layout=widgets.Layout(margin='2px 0'))

form_items = [
    make_row('Gender:', gender),
    make_row('Age:', age),
    make_row('Current S:', currentSmoker),
    make_row('Cigs/Day:', cigsPerDay),
    make_row('BP Meds:', BPMeds),
    make_row('Stroke Hist:', prevalentStroke),
    make_row('Hypertension:', prevalentHyp),
    make_row('Diabetes:', diabetes),
    make_row('Total Chol:', totChol),
    make_row('Systolic BP:', sysBP),
    make_row('Diastolic BP:', diaBP),
    make_row('BMI:', BMI),
    make_row('Heart Rate:', heartRate),
    make_row('Glucose:', glucose),
]

predict_button = widgets.Button(description='Predict')
predict_button.add_class('predict-button')

output_box = widgets.Output()
output_box.layout.width = '320px'
output_box.add_class('output-box')

label_map = {1: "Healthy Heart ❤️", 0: "Unhealthy Heart ⚠️"}

def on_predict_clicked(b):
    with output_box:
        output_box.clear_output()
        try:
            gender_val = 1 if gender.value == 'Male' else 0
            
            input_data = pd.DataFrame({
                'male': [gender_val],
                'age': [age.value],
                'currentSmoker': [currentSmoker.value],
                'cigsPerDay': [cigsPerDay.value],
                'BPMeds': [BPMeds.value],
                'prevalentStroke': [prevalentStroke.value],
                'prevalentHyp': [prevalentHyp.value],
                'diabetes': [diabetes.value],
                'totChol': [totChol.value],
                'sysBP': [sysBP.value],
                'diaBP': [diaBP.value],
                'BMI': [BMI.value],
                'heartRate': [heartRate.value],
                'glucose': [glucose.value],
            })

            row = input_data.iloc[0]

            causes = []
            if row['totChol'] > 240:
                causes.append("high cholesterol")
            if row['cigsPerDay'] > 20:
                causes.append("smoking")
            if row['sysBP'] > 130:
                causes.append("high blood pressure(sys)")
            if row['diaBP'] > 90:
                causes.append("high blood pressure(dia)")
            if row['glucose'] > 100:
                causes.append("high glucose")
            if row['currentSmoker'] == 1:
                causes.append("smoking")
            if row['prevalentHyp'] == 1:
                causes.append("hypertension")
            if row['diabetes'] == 1:
                causes.append("diabetes")

            final_prediction = 0 if len(causes) >= 2 else 1
            result = label_map[final_prediction]

            print(f"Prediction: {result}")
            if final_prediction == 0:
                print("Causes Detected:", ", ".join(causes))
                for cause in causes:
                    display_info(cause)
                    
            else:
                print("No major risk factors detected.")
                print("✅ You're doing great! Here are tips to keep your heart healthy:\n")
                for key, value in heart_health_tips.items():
                    print(f"-> {key.capitalize()}: {value}\n")
        except Exception as e:
            print("Error:", e)


predict_button.on_click(on_predict_clicked)

container = widgets.VBox(form_items + [predict_button, output_box])

# Create the title elements
title = widgets.HTML('<div class="title">Heart Health Prediction</div>')
subtitle = widgets.HTML('<div class="subtitle">Please enter the correct information for the below mentioned questions</div>')
container.add_class('custom-container')

display(container)


In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output

cause_info = {
    "high blood pressure(sys)": {
        "precautions": ["Limit sodium intake", "Exercise regularly", "Avoid stress", "Monitor blood pressure regularly"],
        "remedies": ["DASH diet", "Deep breathing exercises", "Maintain healthy weight"],
        "medications": ["Amlodipine", "Lisinopril", "Losartan"]
    },
    "high blood pressure(dia)": {
        "precautions": ["Limit sodium intake", "Exercise regularly", "Avoid stress", "Monitor blood pressure regularly"],
        "remedies": ["DASH diet", "Deep breathing exercises", "Maintain healthy weight"],
        "medications": ["Amlodipine", "Lisinopril", "Losartan"]
    },
    "smoking": {
        "precautions": ["Avoid tobacco products", "Stay away from passive smoke", "Educate on smoking risks"],
        "remedies": ["Nicotine patches/gum", "Counseling", "Join support groups"],
        "medications": ["Varenicline", "Bupropion", "Nicotine replacement therapy"]
    },
    "diabetes": {
        "precautions": ["Monitor blood sugar", "Maintain healthy diet", "Exercise consistently"],
        "remedies": ["Reduce refined carbs", "Consume more fiber", "Hydrate well"],
        "medications": ["Metformin", "Insulin", "Glipizide"]
    },
    "cholesterol": {
        "precautions": ["Avoid trans fats", "Exercise regularly", "Limit red meat"],
        "remedies": ["Eat oats and legumes", "Use olive oil", "Increase fiber intake"],
        "medications": ["Atorvastatin", "Rosuvastatin", "Ezetimibe"]
    },
    "obesity": {
        "precautions": ["Avoid sugary foods", "Stay active", "Track calorie intake"],
        "remedies": ["Intermittent fasting", "Meal planning", "Join fitness programs"],
        "medications": ["Orlistat", "Liraglutide", "Semaglutide"]
    },
    "stroke history": {
        "precautions": ["Control BP and cholesterol", "Avoid smoking and alcohol", "Follow-up regularly"],
        "remedies": ["Physiotherapy", "Speech therapy", "Healthy lifestyle"],
        "medications": ["Aspirin", "Clopidogrel", "Statins"]
    },
    "high glucose": {
        "precautions": ["Avoid sugar-heavy foods", "Eat low GI foods", "Frequent monitoring"],
        "remedies": ["Cinnamon, fenugreek in diet", "Reduce stress", "Stay hydrated"],
        "medications": ["Metformin", "Insulin", "SGLT2 inhibitors"]
    },
    "high heart rate": {
        "precautions": ["Avoid caffeine", "Manage anxiety", "Stay hydrated"],
        "remedies": ["Meditation", "Slow deep breathing", "Cold splash to face"],
        "medications": ["Beta blockers", "Calcium channel blockers"]
    },
    "age-related risk": {
        "precautions": ["Annual check-ups", "Exercise regularly", "Balanced diet"],
        "remedies": ["Social engagement", "Mental stimulation", "Mobility exercises"],
        "medications": ["As prescribed by physician"]
    },
    "bp medications": {
        "precautions": ["Adhere to schedule", "Don't self-adjust dose", "Monitor side effects"],
        "remedies": ["Lifestyle improvements to reduce dependence", "Avoid alcohol"],
        "medications": ["Amlodipine", "Losartan", "Hydrochlorothiazide"]
    },
    "hypertension": {
        "precautions": ["Adhere to schedule", "Don't self-adjust dose", "Monitor side effects"],
        "remedies": ["Lifestyle improvements to reduce stress"],
        "medications": ["yoga and meditations","Daily walk"]
    },
    "healthy heart": {
        "precautions": ["Daily walk or exercise", "Low-fat diet", "No tobacco or alcohol", "Regular health screenings"],
        "remedies": ["Meditation", "Adequate sleep", "Stay socially active"],
        "medications": ["None – maintain lifestyle and preventive care"]
    }
}

def display_info(cause):
    cause = cause.lower()
    info = cause_info.get(cause)
    if info:
        print(f"\n--- {cause.title()} ---")
        print("\nPrecautions:")
        for item in info["precautions"]:
            print(f"• {item}")
        print("\nRemedies:")
        for item in info["remedies"]:
            print(f"• {item}")
        print("\nMedications:")
        for item in info["medications"]:
            print(f"• {item}")
    else:
        print(f"No information available for {cause}")


In [4]:
heart_health_tips = {
    "diet": "Follow a balanced diet rich in fruits, vegetables, whole grains, lean proteins, and healthy fats (like olive oil and nuts). Avoid excess salt, sugar, and processed foods.",
    "exercise": "Engage in regular physical activity. Aim for at least 150 minutes of moderate aerobic exercise per week, like brisk walking, swimming, or cycling.",
    "hydration": "Drink enough water daily to keep your body and cardiovascular system functioning optimally.",
    "stress": "Practice stress management through mindfulness, yoga, meditation, or hobbies. Chronic stress can increase heart risk.",
    "smoking": "Avoid tobacco products and second-hand smoke. Even occasional smoking harms blood vessels and increases risk.",
    "sleep": "Get 7–9 hours of quality sleep every night. Poor sleep increases the risk of high blood pressure and heart disease.",
    "checkups": "Get regular health checkups to monitor blood pressure, cholesterol, and blood sugar levels, even if you feel fine.",
    "alcohol": "Limit alcohol consumption. Excessive drinking can raise blood pressure and add empty calories.",
    "weight": "Maintain a healthy weight to reduce stress on your heart. Obesity is linked with hypertension, diabetes, and heart disease.",
    "mindset": "Stay positive. A good emotional state is linked with lower heart disease risk. Laugh more, worry less."
}
